# 07 — Realtime Feature Update Job (Modal, every 30 minutes)

This notebook is designed to run **headlessly** (e.g., on Modal) every 30 minutes.

It:
1. Logs into Hopsworks
2. Loads monitoring points from metadata Feature Group
3. Pulls **current** TomTom traffic data for each point
4. Inserts raw rows into `traffic_flow_fg` (10-minute bucket `ts_10m`)
5. Computes the **same engineered features** as your historical backfill notebook and inserts them into `traffic_temporal_fg`
6. Updates hourly **weather** features from Open-Meteo and hourly **TfL disruptions** features (optional)

### Notes on consistency
Your historical engineering uses rolling windows defined in **number of rows** (10-min buckets).
If you only fetch TomTom every 30 minutes, you will have fewer observations and the *effective time span*
of a “window=3” rolling feature will become ~90 minutes.

If you need strict *time-based* window semantics (e.g., exactly 30/60/120 minutes), you should either:
- collect every 10 minutes, OR
- redefine rolling features in minutes and backfill them consistently.

For now, this notebook reproduces the **same feature definitions** as in the historical code.


In [1]:
# If you run this notebook outside an environment that already has dependencies,
# uncomment the following line:
# !pip install -U hopsworks pandas numpy requests

import os
import json
import time
from datetime import datetime, timezone
from typing import Dict, List, Optional

import numpy as np
import pandas as pd
import requests
import hopsworks

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 200)


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1) Configuration

In [2]:
# -------------------------
# Run time
# -------------------------
# For Modal/papermill runs you can pass RUN_TS_UTC="2026-01-09T12:30:00Z"
RUN_TS_UTC = os.getenv("RUN_TS_UTC", "")
if RUN_TS_UTC:
    run_ts = pd.to_datetime(RUN_TS_UTC, utc=True, errors="raise")
else:
    run_ts = pd.Timestamp.now(tz="UTC")

# Use the same 10-minute bucketing as your raw Feature Group
ts_10m = run_ts.floor("10min")

# Hourly buckets for weather and TfL
weather_time_utc = ts_10m.floor("H")
tfl_time_utc = ts_10m.floor("H")

print("run_ts:", run_ts)
print("ts_10m:", ts_10m)
print("weather_time_utc:", weather_time_utc)
print("tfl_time_utc:", tfl_time_utc)

# -------------------------
# Hopsworks connection
# -------------------------
HOPSWORKS_HOST = os.getenv("HOPSWORKS_HOST", "")
HOPSWORKS_PROJECT = os.getenv("HOPSWORKS_PROJECT", "")
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY", "")

# -------------------------
# API Keys
# -------------------------
TOMTOM_API_KEY = os.getenv("TOMTOM_API_KEY", "")
TFL_APP_ID = os.getenv("TFL_APP_ID", "")
TFL_APP_KEY = os.getenv("TFL_APP_KEY", "")

# -------------------------
# Feature Groups (defaults aligned with your existing notebooks)
# -------------------------
RAW_FG_NAME = os.getenv("RAW_FG_NAME", "traffic_flow_fg")
RAW_FG_VERSION = int(os.getenv("RAW_FG_VERSION", "1"))

ENGINEERED_FG_NAME = os.getenv("ENGINEERED_FG_NAME", "traffic_temporal_fg")
ENGINEERED_FG_VERSION = int(os.getenv("ENGINEERED_FG_VERSION", "1"))

METADATA_FG_NAME = os.getenv("METADATA_FG_NAME", "traffic_points_metadata")
METADATA_FG_VERSION = int(os.getenv("METADATA_FG_VERSION", "1"))

WEATHER_FG_NAME = os.getenv("WEATHER_FG_NAME", "weather_10m_fg")
WEATHER_FG_VERSION = int(os.getenv("WEATHER_FG_VERSION", "1"))

TFL_FG_NAME = os.getenv("TFL_FG_NAME", "tfl_disruptions_10m_fg")
TFL_FG_VERSION = int(os.getenv("TFL_FG_VERSION", "1"))

# -------------------------
# Feature engineering params (match your backfill defaults)
# -------------------------
LOW_CONF_THRESHOLD = float(os.getenv("LOW_CONF_THRESHOLD", "0.7"))
ROLL_WINDOWS = [3, 6, 12]  # designed for 10-min buckets

# -------------------------
# Controls
# -------------------------
INSERT_RAW = os.getenv("INSERT_RAW", "1") == "1"
INSERT_ENGINEERED = os.getenv("INSERT_ENGINEERED", "1") == "1"
UPDATE_WEATHER = os.getenv("UPDATE_WEATHER", "1") == "1"
UPDATE_TFL = os.getenv("UPDATE_TFL", "1") == "1"

LOOKBACK_HOURS = int(os.getenv("LOOKBACK_HOURS", "24"))  # used to compute rolling features reliably

assert TOMTOM_API_KEY, "Missing TOMTOM_API_KEY env var"



run_ts: 2026-01-11 13:17:03.077481+00:00
ts_10m: 2026-01-11 13:10:00+00:00
weather_time_utc: 2026-01-11 13:00:00+00:00
tfl_time_utc: 2026-01-11 13:00:00+00:00


## 2) Login to Hopsworks

In [3]:
project = hopsworks.login(
    host="eu-west.cloud.hopsworks.ai",
    project="London_traffic"
)
fs = project.get_feature_store()
raw_fg = fs.get_feature_group(name=RAW_FG_NAME, version=RAW_FG_VERSION)
engineered_fg = fs.get_feature_group(name=ENGINEERED_FG_NAME, version=ENGINEERED_FG_VERSION)
meta_fg = fs.get_feature_group(name=METADATA_FG_NAME, version=METADATA_FG_VERSION)

print("Connected. Feature store:", fs.name)


2026-01-11 14:17:05,863 INFO: Initializing external client
2026-01-11 14:17:05,863 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2026-01-11 14:17:07,138 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/3209
Connected. Feature store: london_traffic_featurestore


## 3) Load monitoring points (metadata)

In [4]:
meta_df = meta_fg.read()
required = {"point_id", "latitude", "longitude"}
missing = required - set(meta_df.columns)
if missing:
    raise ValueError(f"Metadata FG missing required columns: {missing}")

points = meta_df[["point_id", "latitude", "longitude"]].copy()
points["point_id"] = points["point_id"].astype(str)
points["latitude"] = pd.to_numeric(points["latitude"], errors="coerce")
points["longitude"] = pd.to_numeric(points["longitude"], errors="coerce")
points = points.dropna(subset=["point_id", "latitude", "longitude"]).reset_index(drop=True)

print("Loaded points:", len(points))
points.head()


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.26s) 
Loaded points: 200


,point_id,latitude,longitude
0,8618,51.509307,-0.084878
1,16756,51.383489,-0.105944
2,38815,51.393451,0.029589
3,6458,51.573103,-0.212077
4,38022,51.589213,0.270734


## 4) Fetch TomTom Flow data for all points

In [5]:
MAX_POINTS = 50

points_subset = (
    points
    .dropna(subset=["latitude", "longitude"])
    .head(MAX_POINTS)
    .reset_index(drop=True)
)

def fetch_tomtom_flow(lat: float, lon: float, *, zoom: int = 10, unit: str = "kmph") -> Dict:
    # TomTom Flow Segment Data API (v4)
    url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/{zoom}/json"
    params = {
        "key": TOMTOM_API_KEY,
        "point": f"{lat},{lon}",
        "unit": unit,
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

def parse_tomtom_payload(payload: Dict) -> Dict:
    fsd = payload.get("flowSegmentData", {}) if isinstance(payload, dict) else {}
    frc = fsd.get("frc")
    current_speed = fsd.get("currentSpeed")
    free_flow_speed = fsd.get("freeFlowSpeed")
    current_tt = fsd.get("currentTravelTime")
    free_flow_tt = fsd.get("freeFlowTravelTime")
    confidence = fsd.get("confidence")
    road_closure = fsd.get("roadClosure")

    # Derived
    speed_ratio = None
    delay_seconds = None
    try:
        if current_speed is not None and free_flow_speed:
            speed_ratio = float(current_speed) / float(free_flow_speed)
    except Exception:
        speed_ratio = None

    try:
        if current_tt is not None and free_flow_tt is not None:
            delay_seconds = float(current_tt) - float(free_flow_tt)
    except Exception:
        delay_seconds = None

    return {
        "current_speed": float(current_speed) if current_speed is not None else np.nan,
        "free_flow_speed": float(free_flow_speed) if free_flow_speed is not None else np.nan,
        "frc": str(frc) if frc is not None else np.nan,
        "current_travel_time": float(current_tt) if current_tt is not None else np.nan,
        "free_flow_travel_time": float(free_flow_tt) if free_flow_tt is not None else np.nan,
        "confidence": float(confidence) if confidence is not None else np.nan,
        "road_closure": bool(road_closure) if road_closure is not None else False,
        "speed_ratio": float(speed_ratio) if speed_ratio is not None else np.nan,
        "delay_seconds": float(delay_seconds) if delay_seconds is not None else np.nan,
    }

rows = []
errors = 0

for _, p in points_subset.iterrows():
    pid = str(p["point_id"])
    lat = float(p["latitude"])
    lon = float(p["longitude"])
    try:
        payload = fetch_tomtom_flow(lat, lon)
        parsed = parse_tomtom_payload(payload)
        rows.append({
            "point_id": pid,
            "timestamp_utc": run_ts,
            "ts_10m": ts_10m,
            **parsed,
        })
    except Exception as e:
        errors += 1
        rows.append({
            "point_id": pid,
            "timestamp_utc": run_ts,
            "ts_10m": ts_10m,
            "error": str(e),
        })

raw_tick_df = pd.DataFrame(rows)

print("Raw tick df shape:", raw_tick_df.shape, "errors:", errors)

raw_tick_df.head()


Raw tick df shape: (50, 12) errors: 0


,point_id,timestamp_utc,ts_10m,current_speed,free_flow_speed,frc,current_travel_time,free_flow_travel_time,confidence,road_closure,speed_ratio,delay_seconds
0,8618,2026-01-11 13:17:03.077481+00:00,2026-01-11 13:10:00+00:00,23.0,29.0,FRC2,572.0,454.0,1.0,False,0.793103,118.0
1,16756,2026-01-11 13:17:03.077481+00:00,2026-01-11 13:10:00+00:00,6.0,19.0,FRC3,149.0,47.0,1.0,False,0.315789,102.0
2,38815,2026-01-11 13:17:03.077481+00:00,2026-01-11 13:10:00+00:00,33.0,43.0,FRC3,49.0,38.0,1.0,False,0.767442,11.0
3,6458,2026-01-11 13:17:03.077481+00:00,2026-01-11 13:10:00+00:00,50.0,58.0,FRC2,169.0,146.0,1.0,False,0.862069,23.0
4,38022,2026-01-11 13:17:03.077481+00:00,2026-01-11 13:10:00+00:00,108.0,108.0,FRC0,132.0,132.0,1.0,False,1.000000,0.0


## 5) Insert RAW tick into `traffic_flow_fg` (optional, recommended)

In [6]:
if INSERT_RAW:
    # Keep only columns expected by your raw FG schema. If schema differs, adjust here.
    # We also ensure road_closure is numpy bool (not pandas nullable boolean).
    keep_cols = [
        "point_id", "timestamp_utc", "ts_10m", "frc",
        "current_speed", "free_flow_speed", "current_travel_time", "free_flow_travel_time",
        "confidence", "road_closure", "speed_ratio", "delay_seconds"
    ]
    missing = [c for c in keep_cols if c not in raw_tick_df.columns]
    if missing:
        raise ValueError(f"raw_tick_df missing columns: {missing}")

    df_ins = raw_tick_df[keep_cols].copy()
    df_ins["point_id"] = df_ins["point_id"].astype(str)
    df_ins["timestamp_utc"] = pd.to_datetime(df_ins["timestamp_utc"], utc=True)
    df_ins["ts_10m"] = pd.to_datetime(df_ins["ts_10m"], utc=True)

    # Fix Avro bool edge case
    df_ins["road_closure"] = df_ins["road_closure"].fillna(False).astype(bool)

    # Deduplicate on PK
    df_ins = df_ins.drop_duplicates(subset=["point_id", "ts_10m"], keep="last")

    # Avoid inserting duplicates if rerun
    existing_raw = raw_fg.read()
    existing_raw["ts_10m"] = pd.to_datetime(existing_raw["ts_10m"], utc=True, errors="coerce")
    existing_keys = set(zip(existing_raw["point_id"].astype(str), existing_raw["ts_10m"]))
    df_ins["__key"] = list(zip(df_ins["point_id"].astype(str), df_ins["ts_10m"]))
    df_ins = df_ins[~df_ins["__key"].isin(existing_keys)].drop(columns="__key")

    print("Raw rows to insert:", len(df_ins))
    if len(df_ins) > 0:
        raw_fg.insert(df_ins)
        print("Inserted raw tick.")
    else:
        print("No new raw rows (already present).")
else:
    print("Skipping RAW insert (INSERT_RAW=0).")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.62s) 
Raw rows to insert: 50


Uploading Dataframe: 100.00% |██████████| Rows 50/50 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: traffic_flow_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3209/jobs/named/traffic_flow_fg_1_offline_fg_materialization/executions
Inserted raw tick.


## 6) Compute engineered features for this tick and insert into `traffic_temporal_fg`

In [7]:
def time_interval(hour: int) -> str:
    if 7 <= hour < 10:
        return "morning_peak"
    elif 10 <= hour < 16:
        return "midday"
    elif 16 <= hour < 19:
        return "evening_peak"
    else:
        return "night"

def build_engineered_features(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()

    # Ensure types
    df["point_id"] = df["point_id"].astype(str)
    df["ts_10m"] = pd.to_datetime(df["ts_10m"], utc=True, errors="coerce")
    df["timestamp_utc"] = pd.to_datetime(df.get("timestamp_utc", df["ts_10m"]), utc=True, errors="coerce")

    # Sort + dedupe
    df = df.dropna(subset=["point_id", "ts_10m"])
    df = df.sort_values(["point_id", "ts_10m"]).drop_duplicates(subset=["point_id", "ts_10m"], keep="last")

    # Temporal features (same as your backfill)
    df["day_of_week"] = df["ts_10m"].dt.weekday
    df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)

    df["hour"] = df["ts_10m"].dt.hour
    df["minute"] = df["ts_10m"].dt.minute

    df["is_rush_hour"] = ((df["hour"].between(7, 9)) | (df["hour"].between(16, 18))).astype(int)

    df["time_interval"] = df["hour"].apply(time_interval)
    dummies = pd.get_dummies(df["time_interval"], prefix="ti")

    # Ensure stable dummy columns
    for col in ["ti_morning_peak", "ti_midday", "ti_evening_peak", "ti_night"]:
        if col not in dummies.columns:
            dummies[col] = 0

    df = pd.concat([df.drop(columns=["time_interval"]), dummies[["ti_morning_peak","ti_midday","ti_evening_peak","ti_night"]]], axis=1)
    #change to bool
    df["ti_morning_peak"] = df["ti_morning_peak"].astype(bool)
    df["ti_midday"] = df["ti_midday"].astype(bool)
    df["ti_evening_peak"] = df["ti_evening_peak"].astype(bool)
    df["ti_night"] = df["ti_night"].astype(bool)
    # Traffic features (same as your backfill)
    eps = 1e-6
    df["speed_diff"] = df["free_flow_speed"] - df["current_speed"]
    df["travel_time_ratio"] = df["current_travel_time"] / (df["free_flow_travel_time"] + eps)
    df["low_confidence_flag"] = (df["confidence"] < LOW_CONF_THRESHOLD).astype(int)
    df["travel_time_ratio"] = df["travel_time_ratio"].clip(lower=0, upper=10)

    # Rolling features over rows (same as your backfill)
    for w in ROLL_WINDOWS:
        df[f"speed_roll_mean_{w}"] = (
            df.groupby("point_id")["current_speed"]
              .rolling(window=w, min_periods=1)
              .mean()
              .reset_index(level=0, drop=True)
        )
        df[f"speed_roll_std_{w}"] = (
            df.groupby("point_id")["current_speed"]
              .rolling(window=w, min_periods=1)
              .std()
              .reset_index(level=0, drop=True)
              .fillna(0.0)
        )
        df[f"delay_roll_mean_{w}"] = (
            df.groupby("point_id")["delay_seconds"]
              .rolling(window=w, min_periods=1)
              .mean()
              .reset_index(level=0, drop=True)
        )

    return df

if INSERT_ENGINEERED:
    # Read history for lookback
    hist = raw_fg.read()
    hist["ts_10m"] = pd.to_datetime(hist["ts_10m"], utc=True, errors="coerce")
    hist["timestamp_utc"] = pd.to_datetime(hist.get("timestamp_utc", hist["ts_10m"]), utc=True, errors="coerce")
    hist["point_id"] = hist["point_id"].astype(str)

    lookback_start = ts_10m - pd.Timedelta(hours=LOOKBACK_HOURS)
    hist = hist[hist["ts_10m"] >= lookback_start].copy()

    # Append current tick 
    current_raw = raw_tick_df.copy()
    # Keep only required raw columns for feature engineering
    needed = [
        "point_id", "timestamp_utc", "ts_10m", "frc",
        "current_speed", "free_flow_speed", "current_travel_time", "free_flow_travel_time",
        "confidence", "road_closure", "speed_ratio", "delay_seconds"
    ]
    current_raw = current_raw[needed].copy()
    current_raw["timestamp_utc"] = pd.to_datetime(current_raw["timestamp_utc"], utc=True)
    current_raw["ts_10m"] = pd.to_datetime(current_raw["ts_10m"], utc=True)

    combined = pd.concat([hist, current_raw], ignore_index=True)
    combined = combined.drop_duplicates(subset=["point_id", "ts_10m"], keep="last")
    combined = combined.sort_values(["point_id", "ts_10m"]).reset_index(drop=True)

    eng_all = build_engineered_features(combined)

    # Keep only current tick rows
    eng_tick = eng_all[eng_all["ts_10m"] == ts_10m].copy()
    eng_tick = eng_tick.drop_duplicates(subset=["point_id", "ts_10m"], keep="last")

    # Align columns to FG schema (important!)
    schema_cols = [f.name for f in engineered_fg.features]
    for c in schema_cols:
        if c not in eng_tick.columns:
            eng_tick[c] = np.nan

    eng_tick = eng_tick[schema_cols].copy()

    # Avoid duplicates if rerun
    existing_eng = engineered_fg.read()
    existing_eng["ts_10m"] = pd.to_datetime(existing_eng["ts_10m"], utc=True, errors="coerce")
    existing_keys = set(zip(existing_eng["point_id"].astype(str), existing_eng["ts_10m"]))
    eng_tick["__key"] = list(zip(eng_tick["point_id"].astype(str), eng_tick["ts_10m"]))
    eng_tick = eng_tick[~eng_tick["__key"].isin(existing_keys)].drop(columns="__key")

    print("Engineered rows to insert:", len(eng_tick))
    if len(eng_tick) > 0:
        engineered_fg.insert(eng_tick)
        print("Inserted engineered tick.")
    else:
        print("No new engineered rows (already present).")
else:
    print("Skipping engineered insert (INSERT_ENGINEERED=0).")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.71s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.82s) 
Engineered rows to insert: 50


Uploading Dataframe: 100.00% |██████████| Rows 50/50 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: traffic_temporal_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3209/jobs/named/traffic_temporal_fg_1_offline_fg_materialization/executions
Inserted engineered tick.


## 7) Update hourly Weather features from Open-Meteo (optional)

In [8]:
def fetch_openmeteo_hourly(lat: float, lon: float, hour_utc: pd.Timestamp) -> Dict[str, float]:
    url = "https://api.open-meteo.com/v1/forecast"
    hourly_vars = ["temperature_2m", "precipitation", "rain", "snowfall"]
    params = {
        "latitude": lat,
        "longitude": lon,
        "hourly": ",".join(hourly_vars),
        "timezone": "UTC",
        "past_days": 1,
    }
    r = requests.get(url, params=params, timeout=60)
    r.raise_for_status()
    js = r.json()

    times = js.get("hourly", {}).get("time", [])
    if not times:
        raise RuntimeError("Open-Meteo returned no hourly times")

    t = pd.to_datetime(times, utc=True, errors="coerce")
    idx = int(np.argmin(np.abs((t - hour_utc).total_seconds())))

    out = {}
    for v in hourly_vars:
        arr = js.get("hourly", {}).get(v, [])
        out[v] = float(arr[idx]) if idx < len(arr) and arr[idx] is not None else np.nan
    return out


def expand_hour_to_10m_timestamps(hour_utc: pd.Timestamp) -> list[pd.Timestamp]:
    hour_utc = pd.to_datetime(hour_utc, utc=True, errors="coerce").floor("H")
    offsets = pd.to_timedelta([0, 10, 20, 30, 40, 50], unit="m")
    return [hour_utc + off for off in offsets]


if UPDATE_WEATHER:
    weather_10m_fg = fs.get_feature_group(name="weather_10m_fg", version=1)

    weather_time_utc = pd.to_datetime(weather_time_utc, utc=True, errors="coerce").floor("H")
    ts_10m_list = expand_hour_to_10m_timestamps(weather_time_utc)

    # Existing keys only for this hour window
    existing_weather = weather_10m_fg.read(read_options={"use_hive": True})
    existing_weather["ts_10m"] = pd.to_datetime(existing_weather["ts_10m"], utc=True, errors="coerce")
    existing_weather["point_id"] = existing_weather["point_id"].astype(str)
    existing_weather = existing_weather[existing_weather["ts_10m"].isin(ts_10m_list)]
    existing_keys = set(zip(existing_weather["point_id"], existing_weather["ts_10m"]))

    weather_rows = []
    for _, p in points.iterrows():
        pid = str(p["point_id"])
        lat = float(p["latitude"])
        lon = float(p["longitude"])

        try:
            vals = fetch_openmeteo_hourly(lat, lon, weather_time_utc)

            for ts_10m in ts_10m_list:
                key = (pid, ts_10m)
                if key in existing_keys:
                    continue
                weather_rows.append({
                    "point_id": pid,
                    "ts_10m": ts_10m,
                    "precipitation": vals.get("precipitation", np.nan),
                    "rain": vals.get("rain", np.nan),
                    "snowfall": vals.get("snowfall", np.nan),
                    "temperature_2m": vals.get("temperature_2m", np.nan),
                })

        except Exception:
            # No error column in schema: insert NaNs
            for ts_10m in ts_10m_list:
                key = (pid, ts_10m)
                if key in existing_keys:
                    continue
                weather_rows.append({
                    "point_id": pid,
                    "ts_10m": ts_10m,
                    "precipitation": np.nan,
                    "rain": np.nan,
                    "snowfall": np.nan,
                    "temperature_2m": np.nan,
                })

    weather_df_new = pd.DataFrame(weather_rows)

    if len(weather_df_new) > 0:
        # Ensure correct column order
        weather_df_new["point_id"] = weather_df_new["point_id"].astype(str)
        weather_df_new["ts_10m"] = pd.to_datetime(weather_df_new["ts_10m"], utc=True, errors="coerce")

        weather_df_new = weather_df_new[
            ["point_id", "ts_10m", "precipitation", "rain", "snowfall", "temperature_2m"]
        ]

        print("Weather 10-min rows to insert:", len(weather_df_new))
        weather_10m_fg.insert(weather_df_new, write_options={"wait_for_job": True})
        print("Inserted weather_10m for hour:", weather_time_utc)
    else:
        print("No new weather_10m rows for this hour window (already present).")
else:
    print("Skipping weather update (UPDATE_WEATHER=0).")


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.74s) 
No new weather_10m rows for this hour window (already present).


## 8) Update hourly TfL disruption features (optional)

In [9]:
import requests
from typing import Optional

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1 = np.radians(lat1); lon1 = np.radians(lon1)
    lat2 = np.radians(lat2); lon2 = np.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    c = 2*np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R*c

def extract_point_latlon(point_obj) -> Optional[tuple]:
    if not isinstance(point_obj, dict):
        return None
    lat = point_obj.get("lat", None)
    lon = point_obj.get("lon", None)
    if lat is None or lon is None:
        lat = point_obj.get("latitude", None)
        lon = point_obj.get("longitude", None)
    try:
        if lat is None or lon is None:
            return None
        return (float(lat), float(lon))
    except Exception:
        return None

def fetch_tfl_active_road_disruptions() -> pd.DataFrame:
    url = "https://api.tfl.gov.uk/Road/all/Disruption"
    params = {
        "stripContent": True,
        "closures": True,
    }
    if TFL_APP_ID:
        params["app_id"] = TFL_APP_ID
    if TFL_APP_KEY:
        params["app_key"] = TFL_APP_KEY

    r = requests.get(url, params=params, timeout=60)
    if r.status_code >= 400:
        raise RuntimeError(f"TfL {r.status_code}: {r.text[:500]}")
    r.raise_for_status()
    raw = r.json()
    return pd.DataFrame(raw if isinstance(raw, list) else [])

RADIUS_KM = float(os.getenv("TFL_RADIUS_KM", "0.5"))

def expand_hour_to_10m_timestamps(hour_utc: pd.Timestamp) -> list[pd.Timestamp]:
    hour_utc = pd.to_datetime(hour_utc, utc=True, errors="coerce").floor("H")
    offsets = pd.to_timedelta([0, 10, 20, 30, 40, 50], unit="m")
    return [hour_utc + off for off in offsets]


if UPDATE_TFL:
    if not TFL_APP_KEY and not TFL_APP_ID:
        print("Skipping TfL update because TFL_APP_KEY/TFL_APP_ID are not set.")
    else:
        # IMPORTANT: point to your 10-min FG
        tfl_10m_fg = fs.get_feature_group(name="tfl_disruptions_10m_fg", version=1)

        # Define hour anchor (hourly source)
        hour_start = pd.to_datetime(tfl_time_utc, utc=True, errors="coerce").floor("H")
        hour_end = hour_start + pd.Timedelta(hours=1)

        ts_10m_list = expand_hour_to_10m_timestamps(hour_start)

        # Points
        pts = points.copy()
        pts["point_id"] = pts["point_id"].astype(str)
        pts = pts.dropna(subset=["latitude", "longitude"]).reset_index(drop=True)

        # Fetch current disruptions
        raw_df = fetch_tfl_active_road_disruptions()

        # If nothing returned -> zeros for all points (replicated to 10-min)
        if raw_df.empty:
            base_out = pd.DataFrame({
                "point_id": pts["point_id"],
                "disruption_count": 0,
                "is_works": 0,
                "is_incident": 0,
                "is_active": 0,
                "max_ordinal": 0,
            })
        else:
            # Parse disruption times
            raw_df["start"] = pd.to_datetime(raw_df.get("startDateTime"), utc=True, errors="coerce")
            raw_df["end"] = pd.to_datetime(raw_df.get("endDateTime"), utc=True, errors="coerce")
            raw_df["end_filled"] = raw_df["end"]
            raw_df.loc[raw_df["end_filled"].isna(), "end_filled"] = raw_df["start"] + pd.Timedelta(hours=1)

            # Keep disruptions overlapping this hour window
            raw_df = raw_df[
                (raw_df["start"].isna()) | ((raw_df["start"] < hour_end) & (raw_df["end_filled"] >= hour_start))
            ].copy()

            # Extract disruption point coords
            coords = raw_df.get("point", pd.Series([None]*len(raw_df))).apply(extract_point_latlon)
            raw_df["d_lat"] = coords.apply(lambda x: x[0] if x else np.nan)
            raw_df["d_lon"] = coords.apply(lambda x: x[1] if x else np.nan)
            raw_df = raw_df.dropna(subset=["d_lat", "d_lon"]).reset_index(drop=True)

            if raw_df.empty:
                base_out = pd.DataFrame({
                    "point_id": pts["point_id"],
                    "disruption_count": 0,
                    "is_works": 0,
                    "is_incident": 0,
                    "is_active": 0,
                    "max_ordinal": 0,
                })
            else:
                # Cross join disruptions x points (works well for ~50-200 points)
                cross = pts.rename(columns={"latitude": "p_lat", "longitude": "p_lon"}).copy()
                cross["key"] = 1
                raw_df["key"] = 1
                cross = cross.merge(raw_df, on="key").drop(columns=["key"])

                # Distance filter
                cross["dist_km"] = haversine_km(cross["p_lat"], cross["p_lon"], cross["d_lat"], cross["d_lon"])
                cross = cross[cross["dist_km"] <= RADIUS_KM].copy()

                if cross.empty:
                    base_out = pd.DataFrame({
                        "point_id": pts["point_id"],
                        "disruption_count": 0,
                        "is_works": 0,
                        "is_incident": 0,
                        "is_active": 0,
                        "max_ordinal": 0,
                    })
                else:
                    cross["disruption_count"] = 1
                    cross["is_works"] = (cross.get("category").astype(str) == "Works").astype(int)
                    cross["is_incident"] = (cross.get("category").astype(str) == "Incident").astype(int)
                    cross["is_active"] = (cross.get("status").astype(str).str.lower() == "active").astype(int)
                    cross["max_ordinal"] = pd.to_numeric(cross.get("ordinal"), errors="coerce").fillna(0)

                    base_out = (
                        cross.groupby("point_id", as_index=False)
                        .agg(
                            disruption_count=("disruption_count", "sum"),
                            is_works=("is_works", "max"),
                            is_incident=("is_incident", "max"),
                            is_active=("is_active", "max"),
                            max_ordinal=("max_ordinal", "max"),
                        )
                    )

                    # Ensure all points exist (fill zeros)
                    base_out = base_out.merge(pd.DataFrame({"point_id": pts["point_id"]}), on="point_id", how="right")
                    base_out = base_out.fillna(0)

        # Replicate to 10-min timestamps
        out_list = []
        for ts_10m in ts_10m_list:
            tmp = base_out.copy()
            tmp["ts_10m"] = ts_10m
            out_list.append(tmp)

        out = pd.concat(out_list, ignore_index=True)

        # Type fixes for bigint columns
        for c in ["disruption_count", "is_works", "is_incident", "is_active", "max_ordinal"]:
            if c in out.columns:
                out[c] = pd.to_numeric(out[c], errors="coerce").fillna(0).astype("int64")

        out["point_id"] = out["point_id"].astype(str)
        out["ts_10m"] = pd.to_datetime(out["ts_10m"], utc=True, errors="coerce")

        # Align to schema (exact order)
        out = out[[
            "point_id", "ts_10m",
            "disruption_count", "is_active", "is_incident", "is_works", "max_ordinal"
        ]]

        print("TfL 10-min rows to upsert:", len(out))
        tfl_10m_fg.insert(out, write_options={"upsert": True})
        print("Upserted TfL 10-min rows for hour:", hour_start)
else:
    print("Skipping TfL update (UPDATE_TFL=0).")


TfL 10-min rows to upsert: 1200


Uploading Dataframe: 100.00% |██████████| Rows 1200/1200 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: tfl_disruptions_10m_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://eu-west.cloud.hopsworks.ai:443/p/3209/jobs/named/tfl_disruptions_10m_fg_1_offline_fg_materialization/executions
2026-01-11 14:23:59,409 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-11 14:24:02,567 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-11 14:26:18,732 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2026-01-11 14:26:21,907 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2026-01-11 14:26:22,713 INFO: Waiting for log aggregation to finish.
2026-01-11 14:26:22,714 INFO: Execution finished successfully.
Upserted TfL 10-min rows for hour: 2026-01-11 13:00:00+00:00


## 9) Done